In [1]:
%load_ext cython

In [15]:
%%cython -a
import cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)


def transition_mat(double[:,:] A, double[:] inpt, int kbar):
    cdef double b= inpt[0]
    cdef double gamma_kbar = inpt[2]
    #Not sure If it is double[:] or double[:,:] because it is a column vector
    cdef double[:,:] gamma = np.zeros((kbar,1))
    cdef int kbar2 = 2**kbar
    cdef double[:,:] prob = np.ones((kbar2,1))
    
    cdef int i
    cdef int j
    cdef int m
    
    
    gamma[0,0] = 1-(1-gamma_kbar)**(1/(b**(kbar-1)))
    for i in range(1,kbar):
        gamma[i,0] = 1-(1-gamma[0,0])**(b**(i))
    
    #Not sure if I have to use "double[:,:]" to redefine gamma 
    for i in range(kbar):
        gamma[i,0] = gamma[i,0]*0.5
    gamma = np.c_[gamma,gamma]
    for i in range(kbar):
        gamma[i,0] = 1-gamma[i,1]
    
    
    for i in range(kbar2):
        for m in range(kbar):
            prob[i,0] =prob[i,0] * gamma[kbar-m-1,
                np.unpackbits(np.array([i],dtype = np.uint8))[-(m+1)]]
            
    for i in range(2**(kbar-1)):
        for j in range(i,2**(kbar-1)):
            A[kbar2-i-1,j] = prob[np.rint(kbar2 - A[i,j]-1).astype(int),0]
            A[kbar2-j-1,i] = A[kbar2-i-1,j]
            A[j,kbar2-i-1] = A[kbar2-i-1,j]
            A[i,kbar2-j-1] = A[kbar2-i-1,j]
            A[i,j] = prob[np.rint(A[i,j]).astype(int),0]
            A[j,i] = A[i,j].copy()
            A[kbar2-j-1,kbar2-i-1] = A[i,j]
            A[kbar2-i-1,kbar2-j-1] = A[i,j]
        
    return(A)

DistutilsPlatformError: Unable to find vcvarsall.bat

In [4]:
%%cython -a
import numpy as np
def transition_mat(A,inpt,kbar):
    b = inpt[0]
    gamma_kbar = inpt[2]
    gamma = np.zeros((kbar,1))
    gamma[0,0] = 1-(1-gamma_kbar)**(1/(b**(kbar-1)))
    for i in range(1,kbar):
        gamma[i,0] = 1-(1-gamma[0,0])**(b**(i))
    gamma = gamma*0.5
    gamma = np.c_[gamma,gamma]
    gamma[:,0] = 1 - gamma[:,1]
    kbar2 = 2**kbar
    prob = np.ones((kbar2,1))
    
    for i in range(kbar2):
        for m in range(kbar):
            prob[i,0] =prob[i,0] * gamma[kbar-m-1,
                np.unpackbits(np.array([i],dtype = np.uint8))[-(m+1)]]
            
    for i in range(2**(kbar-1)):
        for j in range(i,2**(kbar-1)):
            A[kbar2-i-1,j] = prob[np.rint(kbar2 - A[i,j]-1).astype(int),0]
            A[kbar2-j-1,i] = A[kbar2-i-1,j]
            A[j,kbar2-i-1] = A[kbar2-i-1,j]
            A[i,kbar2-j-1] = A[kbar2-i-1,j]
            A[i,j] = prob[np.rint(A[i,j]).astype(int),0]
            A[j,i] = A[i,j].copy()
            A[kbar2-j-1,kbar2-i-1] = A[i,j]
            A[kbar2-i-1,kbar2-j-1] = A[i,j]
        
    return(A)

DistutilsPlatformError: Unable to find vcvarsall.bat